# Redcross price tagger

Simple price tagger tool for red cross volunteers to put price to pre-love item in the op shop.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as plt

In [2]:
df = pd.read_csv('redcross.csv')

In [3]:
df.head(2)

,web-scraper-order,web-scraper-start-url,itemlink,itemlink-href,name,price,det_brand,de_col,det_good,det_material,det_size,descrp1,descrp2
0,1625052999-207,https://shop.redcross.org.au/preloved/ladies-c...,Gorman Mustard Patterned Mini Skirt$50.00Add t...,https://shop.redcross.org.au/preloved/ladies-c...,Gorman Mustard Patterned Mini Skirt,$50.00,Brand: Gorman,Colour: Yellow,Condition: Good,Material: Linen & Cotton,SecondaryColour: Pink,Cute Gorman yellow/black/beige/pink pattern mi...,Small white smudge near left seam (see images)...
1,1625052554-138,https://shop.redcross.org.au/preloved/ladies-c...,Kate Spade New York Slingback Heels$35.00Add t...,https://shop.redcross.org.au/preloved/ladies-c...,Kate Spade New York Slingback Heels,$35.00,Brand: Kate Spade,Colour: Black,Condition: Good,Material: Patent leather,Size: 10B,Kate Spade black patent leather slingback open...,General wear to bottom of soles


In [4]:
df.det_material.str.split(':').map(lambda x: x[1]).unique()

array([' Linen & Cotton', ' Patent leather', ' Leather/Haircalf',
       ' Linen', ' Leather', ' Wool', ' Polyester',
       ' Cotton Viscose Linen', ' Viscose Rayon', ' 40',
       ' Leather and wood', ' Leather/velvet', ' Cotton',
       ' Bamboo/Polyester', ' Viscose/Polyester', ' Cow hide/leather',
       ' Polyester & Viscose', ' Leather - Lambskin', ' Rayon', ' Suede',
       ' Viscose', ' Silk', ' Acrylic', ' Lyocell', ' Cotton/spandex',
       ' Corduroy', ' Polyester/Elastine', ' Taffeta', ' Vinyl',
       ' Cotton and viscose', ' Unknown', ' Crepe/Polyester', ' Pink',
       ' Red', ' Leather/Wood', ' Leather/Suede', ' Satin and leather',
       ' Modacrylic/acrylic/cotton', ' Rayon/Polyester', ' Synthetic',
       ' Leather/synthetic', ' Tencel', ' Synthethic/leather',
       ' Cotton/Tencel', ' Melflex', ' Cupro', ' Velvet and leather',
       '  S', ' Cotton mix', ' Cotton blend', ' Silk and polyester',
       ' Polyester/Viscose/Silk', ' Faux Leather', ' Wool/ Synthetic',

### Cleaning dataset

In [5]:
df['price'] = df.price.str.replace('$','').astype(float)
df.iloc[:,6:11] = df.iloc[:,6:11].replace(np.nan, ":", regex=True)

In [6]:
#newc = df.iloc[:,6:11].replace(np.nan, ":", regex=True).apply(lambda x: x.str.split(':').map(lambda s:s[0]), axis=1)

In [7]:
def pair2columns(df, init_range = 6, fin_range = 11):
    """
    Takes a panda dataframe columns with key:value pair and turn it into new columns where
    Key is column name and value the column value. Check other columns for aligment issues.
    """
    for c in range(init_range,fin_range):
        types = df.iloc[:,c].str.split(':').map(lambda x: x[0]).unique()
        for t in types:
            proper = [i[1] if i[0] == t else np.nan for i in df.iloc[:,c].str.split(':')]
            if t in df.columns:
                df[t].fillna(pd.Series(proper), inplace=True)
            else:
                df[t] = proper

pair2columns(df)

In [8]:
df[df.Size.isna()]['itemlink-href'][0]

'https://shop.redcross.org.au/preloved/ladies-clothing-and-accessories/ladies-bottoms/gorman-mustard-patterned-mini-skirt/'

### Try to recapture missing data by scraping

In [15]:
import requests
import lxml.html
import re

def missingParts(df,column,href):
    """
    Scrape missing attribute from href website. Input:
    -Pandas dataframe
    -Column with NaN values to retrieve
    -column 
    """
    
    headers = {'User-Agent': 'Mozilla/5.0'}

    missPart = []
    for url in df[column.isna()][href]:
        response = requests.get(url, headers=headers, stream=True)
        response.raw.decode_content = True
        tree = lxml.html.parse(response.raw)
        sc = tree.xpath("//*[contains(.,'Size')]/text()")
        sc = [i for i in sc if '\"Size' in i]
        if sc:
            sc = re.search('\"Size(.*)images\"',sc[0]).group(1)
            sc = re.search('\:(.*)\}',sc).group(1).replace("\\","").replace('\"','')
            missPart.append(sc)
        else:
            missPart.append(np.nan)
        return missPart

df.loc[df.Size.isna(),'Size'] = missingParts(df,df.Size, 'itemlink-href')

## data to model feed

In [349]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
#from nltk import stopwords

x_train, x_test, y_train, y_test = train_test_split(df.name, df.price, random_state=0)

vect = TfidfVectorizer(ngram_range=(1, 3),min_df=5,lowercase=True, stop_words='english').fit(x_train)
x_train_vectorized = vect.transform(x_train)
x_test_vectorized = vect.transform(x_test)

from sklearn.feature_selection import mutual_info_classif

def mutualInfoScores(df, y):
    """
    Return sorted Mutual info score list
    """
    res = zip(range(df.shape[0]),
                   mutual_info_classif(df, y, discrete_features=True)
                   )
    sorted_scores = sorted(res, key= lambda k: k[1], reverse=True)
    return sorted_scores


#Get scores for each feature

scores = mutualInfoScores(x_train_vectorized, y_train)

#x_train_vectorized = x_train_vectorized[:,[i for i,e in scores[:30]]]
#x_test_vectorized = x_test_vectorized[:,[i for i,e in scores[:30]]]

### Clean typos by clustering with levenshtain distance

In [16]:
def norm_len(i):
    """
    Calculate normalization value. The input is a tuple with (word, (sim_word, sim_score)).
    Measure absolute difference between a word lenght and its similar word length, normalized by the word length.
    Solve the problem of high similarity scores given to short words where few letters return high levenshtain similarity values.
    Return 0 < score < 1
    """
    return abs(len(i[0]) - len(i[1][0]))/len(i[0])



def merge_sim(column, normalization=0.4, lev_sim=80):
    """
    calculate levenshtain distance for every word in category. Arguments are:
     - the list of values (pandas column), 
     - normalization value threshold,
     - levenshtain similarity score threshold
    """
    from fuzzywuzzy import process, fuzz
    lev=[]
    unique = list(column.replace(np.nan, " unknown").str.lower().str.replace(' designs','').str.replace(' design','').unique()) # get unique values
    for w in unique:
        options = [i for i in unique if i != w] # exclude current value
        lev.append((w,process.extractOne(w,options, scorer= fuzz.token_set_ratio))) # calculate ratio of the most similar
        
    k = {i[0]:i[1][0] for i in lev if i[1][1] > lev_sim and norm_len(i)< normalization} # filter and turn to dictionary
    # Eliminate reciprocal values by first apparience
    k2 = {}
    for i,j in k.items():
        if j not in k2:
            k2[i]=j
    
    return [k2[i] if i in k2 else i for i in column.str.lower()] #return list with replacements

t = merge_sim(df.Brand)

In [19]:
set(df.Brand)

{' AF Vandervorst',
 ' ALPHA60',
 ' AX Armani Exchange',
 ' Alannah Hill',
 ' Aldo',
 ' Alexander Wang',
 ' Alpha60',
 ' Anne Klein',
 ' Balenciaga Paris',
 ' Bally',
 ' Bardot',
 ' Beau Coops',
 ' Bec + Bridge',
 ' Bettina Liano',
 ' Betty Chow Los Angeles',
 ' Bewitch',
 ' Blak Basics by Theresa Hodges',
 ' Bogner',
 ' Bosari',
 ' Bottega Veneta',
 ' Braun Buffel',
 ' Brunello Cucinelli',
 ' Burberry',
 ' CUE',
 ' Calvin Klein',
 ' Camilla & Marc',
 ' Camilla andMarc',
 ' Car Shoe',
 ' Carla Zampatti',
 ' Caron Callahan',
 ' Carre',
 ' Cesare Paciotti',
 ' Chanel',
 ' Charlie Brown',
 ' Chercher La Femme',
 ' Christian Louboutin',
 ' Coach',
 ' Collette Dinnigan',
 ' Converse',
 ' Cotton On',
 ' Country Road',
 ' Cue',
 ' Cue Design',
 ' DKNY',
 ' David Lawrence',
 ' David Molho Creation',
 ' Decjuba',
 ' Diemme',
 ' Dolce & Gabbana',
 ' Dr Martens',
 ' Elm',
 ' Em & Kikki',
 ' Emporio Armani',
 ' Engmeny',
 ' Fame Agenda',
 ' Feathers',
 ' Fenn Wright Manson',
 ' Fila',
 ' Forbat',


In [20]:
set(t)

{' af vandervorst',
 ' alannah hill',
 ' aldo',
 ' alexander wang',
 ' alpha60',
 ' anne klein',
 ' ax armani exchange',
 ' balenciaga paris',
 ' bally',
 ' bardot',
 ' beau coops',
 ' bec + bridge',
 ' bettina liano',
 ' betty chow los angeles',
 ' bewitch',
 ' blak basics by theresa hodges',
 ' bogner',
 ' bosari',
 ' bottega veneta',
 ' braun buffel',
 ' brunello cucinelli',
 ' burberry',
 ' búl',
 ' calvin klein',
 ' camilla & marc',
 ' car shoe',
 ' carla zampatti',
 ' caron callahan',
 ' carre',
 ' cesare paciotti',
 ' chanel',
 ' charlie brown',
 ' chercher la femme',
 ' christian louboutin',
 ' coach',
 ' collette dinnigan',
 ' converse',
 ' cotton on',
 ' country road',
 ' cue',
 ' cue design',
 ' david lawrence',
 ' david molho creation',
 ' decjuba',
 ' diemme',
 ' dkny',
 ' dolce & gabbana',
 ' dr martens',
 ' eileen kirby',
 ' elm',
 ' em & kikki',
 ' emporio armani',
 ' engmeny',
 ' fame agenda',
 ' feathers',
 ' fenn wright manson',
 ' fila',
 ' forbat',
 ' george gross'

In [585]:
[(norm_len(i),i) for i in lev if i[1][1]>80 and norm_len(i)<0.4]

[(0.06666666666666667, (' camilla & marc', (' camilla andmarc', 89))),
 (0.0625, (' camilla andmarc', (' camilla & marc', 89))),
 (0.0, (' giuseppe zanotti', (' guiseppe zanotti', 94))),
 (0.0, (' guiseppe zanotti', (' giuseppe zanotti', 94))),
 (0.34782608695652173, (' ruby by leona edmiston', (' leona edmiston', 100))),
 (0.10526315789473684, (' scanlan & theodore', (' scanlan theodore', 100))),
 (0.11764705882352941, (' scanlan theodore', (' scanlan & theodore', 100))),
 (0.14285714285714285, ('unknown', (' unknown', 100)))]

In [ ]:
[i for i in k.keys() if i]

In [586]:
k = {i[0]:i[1][0] for i in lev if i[1][1]>80 and norm_len(i)<0.4}
[k[i] if i in k else i for i in df.Brand.str.lower()]

[' gorman',
 ' kate spade',
 ' michael kors',
 ' harare',
 ' michael kors',
 ' tootal',
 ' jimmy choo',
 ' revival',
 ' scanlan & theodore',
 ' sass & bide',
 ' dolce & gabbana',
 ' michael kors',
 ' michael kors',
 ' guido pasquali',
 ' prada',
 ' anne klein',
 ' mimco',
 ' carla zampatti',
 ' viktoria & woods',
 ' gorman',
 ' carla zampatti',
 ' calvin klein',
 ' rabeanco',
 ' sergio rossi',
 ' house of harlow',
 ' gucci',
 ' scanlan theodore',
 ' alannah hill',
 ' david lawrence',
 ' patrizia pepe',
 ' guiseppe zanotti',
 ' tommy hilfiger',
 ' ralph lauren',
 ' louis feraud',
 ' karen walker',
 ' versace',
 ' sass & bide',
 ' scanlan & theodore',
 ' robert clergerie',
 ' skin and threads',
 ' mimco',
 ' saba',
 ' cesare paciotti',
 ' carre',
 ' supertrash',
 " kitten d'amour",
 ' stella luna',
 ' vintage',
 ' ka-he',
 ' sass & bide',
 ' alannah hill',
 ' sportscraft',
 ' camilla & marc',
 ' eileen kirby',
 ' car shoe',
 ' homemade',
 ' bottega veneta',
 ' guess',
 ' alannah hill',
 

### data to dummy and merge

### Regression model development

In [350]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared

kl = KernelRidge(alpha=0.000001, kernel = 'sigmoid')

model1 = kl.fit(x_train_vectorized, y_train)

prediction = model1.predict(x_test_vectorized)

metrics.r2_score(y_test, prediction)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


0.23716746078176343

In [191]:
from sklearn.linear_model import Ridge

lm = Ridge(alpha=0.01)

model1 = lm.fit(x_train_vectorized, y_train)

prediction = model1.predict(x_test_vectorized)

metrics.r2_score(y_test, prediction)

0.19128499896442797

In [346]:
from sklearn.svm import LinearSVR


lm = LinearSVR(C=1.0)

model1 = lm.fit(x_train_vectorized, y_train)

prediction = model1.predict(x_test_vectorized)

metrics.r2_score(y_test, prediction)

-0.11321690679701968

In [355]:
from sklearn.linear_model import Lasso


lm = Lasso(alpha=0.01)

model1 = lm.fit(x_train_vectorized, y_train)

prediction = model1.predict(x_test_vectorized)

metrics.r2_score(y_test, prediction)

0.12006512659229007

In [97]:
print(x_train_vectorized)

  (0, 59)	0.6904505270884232
  (0, 36)	0.7233796165522764
  (1, 48)	0.5511171073669887
  (1, 47)	0.5511171073669887
  (1, 25)	0.5511171073669887
  (1, 24)	0.298009734576455
  (3, 91)	1.0
  (4, 77)	1.0
  (5, 75)	0.41510335619560107
  (5, 37)	0.4992318963126324
  (5, 24)	0.2828281188061528
  (5, 2)	0.4992318963126324
  (5, 1)	0.4992318963126324
  (6, 35)	0.41057714136650814
  (6, 29)	0.45126963158358074
  (6, 23)	0.45126963158358074
  (6, 22)	0.45126963158358074
  (6, 19)	0.46956742840465837
  (7, 77)	0.6407448522945599
  (7, 56)	0.7677538891194383
  (8, 92)	0.3391428833479694
  (8, 76)	0.5049207579804826
  (8, 75)	0.4007222376034265
  (8, 10)	0.46315639829906113
  (8, 0)	0.5049207579804826
  :	:
  (191, 50)	1.0
  (192, 87)	0.33093597479037135
  (192, 82)	0.33093597479037135
  (192, 81)	0.3607775771808935
  (192, 57)	0.3607775771808935
  (192, 40)	0.3607775771808935
  (192, 39)	0.3607775771808935
  (192, 26)	0.3607775771808935
  (192, 18)	0.3607775771808935
  (193, 45)	0.2577316921548463